In [4]:
# repo rootz
%cd /work

/work


In [5]:
!ls 

LICENSE    docker    out    pyproject.toml  scripts  tests
README.md  examples  paper  resources	    src


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [5]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [3]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [4]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [5]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [77]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [17]:
# Fig2 run
#!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/*
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,context_swap,stress \
  --k-claims 50 \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
    --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,context_swap,stress
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 105
[run_fig2] (1/105) HNSC ours gate=hard tau=0.20
[run_fig2] (2/105) HNSC stress gate=hard tau=0.20
[run_fig2] (3/105) HNSC context_swap gate=hard tau=0.20
[run_fig2] (4/105) HNSC ours gate=hard tau=0.40
[run_fig2] (5/105) HNSC stress gate=hard tau=0.40
[run_fig2] (6/105) HNSC context_swap gate=hard tau=0.40
[run_fig2] (7/105) HNSC ours gate=hard tau=0.60
[run_fig2] (8/105) HNSC stress gate=hard tau=0.60
[run_fig2] (9/105) HNSC context_swap gate=hard tau=0.60
[run_fig2] (10/105) HNSC ours gate=hard tau=0.80
[run_fig2] (11/105) HNSC stress gate=hard tau=0.80
[run_

In [18]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root "/work/paper/source_data/PANCAN_TP53_v1/out" \
  --out "/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv"

[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv rows=105
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [20]:
import pandas as pd
df = pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv", sep="\t")
g = df.groupby(["condition","variant"])["tau"].nunique().sort_values()
print(g.head(20))
print("min n_tau =", g.min(), "max n_tau =", g.max(), "rows =", len(df))

condition  variant     
BRCA       context_swap    5
UCEC       context_swap    5
SKCM       stress          5
           ours            5
           context_swap    5
OV         stress          5
           ours            5
           context_swap    5
LUSC       stress          5
UCEC       ours            5
LUSC       ours            5
LUAD       stress          5
           ours            5
           context_swap    5
HNSC       stress          5
           ours            5
           context_swap    5
BRCA       stress          5
           ours            5
LUSC       context_swap    5
Name: tau, dtype: int64
min n_tau = 5 max n_tau = 5 rows = 105


In [116]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_abstain_variants.pdf \
  --ycol abstain_rate_total \
  --variants ours,context_swap,stress \
  --label-mode variant


In [117]:
!python paper/scripts/fig2_lines_status_by_tau.py \
  --in /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/FIg2_HNSC_abstain_vs_tau_variants.pdf \
  --condition HNSC \
  --gate-mode hard \
  --variants ours,context_swap,stress \
  --metric abstain \
  --label-mode proposed


In [76]:
!python paper/scripts/fig2_make_labels_template.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --condition HNSC \
  --per-run 30 \
  --prefer-status PASS \
  --with-hints \
  --rater-id R1 \
  --label-version v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv rows=49
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [55]:
!python paper/scripts/fig2_check_labels_merge.py \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --root   /work/paper/source_data/PANCAN_TP53_v1/out \
  --condition HNSC \
  --prefer-status PASS


[OK] labels claim_id unique
[OK] labels human_label values look valid (or blank)
[OK] audit logs used: 15
[OK] unique claim_id in audit (all statuses): 50
[OK] unique claim_id in audit (status=PASS): 43
[OK] labels rows: 49
[OK] labels claim_id found in audit: 49
[OK] non-blank labels total: 49
[OK] non-blank labels among audit status=PASS: 43
[INFO] fraction of non-blank labels that target status=PASS: 0.878
[DONE] If this is clean, pass --labels to fig2_collect_risk_coverage.py


In [118]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --method ours \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv rows=105
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [119]:
!python paper/scripts/fig2_plot_scatter_human_risk.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_HNSC_scatter_human_risk.pdf \
  --condition HNSC \
  --ycol risk_human_nonaccept \
  --variants ours,context_swap,stress \
  --annotate-tau \
  --annotate-npass


Traceback (most recent call last):
  File "/work/paper/scripts/fig2_plot_scatter_human_risk.py", line 275, in <module>
    main()
  File "/work/paper/scripts/fig2_plot_scatter_human_risk.py", line 156, in main
    raise ValueError(f"No rows match condition={args.condition}")
ValueError: No rows match condition=HNSC


In [133]:
!python paper/scripts/figS_plot_abstain_reasons.py \
  --audit-logs \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.90/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.90/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.90/audit_log.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_HNSC_abstain_reasons_tau0.90.pdf \
  --title "" \
  --topk 5


In [6]:
# 1) fetch
!python paper/scripts/figS_beataml_fetch_inputs.py

[beataml_fetch] downloading counts: https://github.com/biodev/beataml2.0_data/raw/main/beataml_waves1to4_counts_dbgap.txt
[beataml_fetch] downloading mutations: https://github.com/biodev/beataml2.0_data/raw/main/beataml_wes_wv1to4_mutations_dbgap.txt
[beataml_fetch] downloading clinical: https://github.com/biodev/beataml2.0_data/raw/main/beataml_wv1to4_clinical.xlsx
[beataml_fetch] OK
  counts: /work/paper/source_data/BEATAML_TP53_v1/raw/beataml_waves1to4_counts_dbgap.txt  (sha256=26302116e26138e980a7816303a0886a89a3c951f3ba673051a46c96a658680d)
  mutations: /work/paper/source_data/BEATAML_TP53_v1/raw/beataml_wes_wv1to4_mutations_dbgap.txt  (sha256=5a5a5eb8f492b1385aebe85c490b9333f65590f09391a7c1951b04dd1dba1680)
  clinical: /work/paper/source_data/BEATAML_TP53_v1/raw/beataml_wv1to4_clinical.xlsx  (sha256=bc692f647f93945e1cf883271af5501bf75c8af3e681676241093c198ed167ad)


In [10]:
# 2) groups
#!pip install openpyxl
!python paper/scripts/figS_beataml_make_groups.py

[beataml_groups] counts: n_samples=710 head=['BA2000R', 'BA2003R', 'BA2004R', 'BA2011R', 'BA2019R']
[beataml_groups] mutations: shape=(11721, 32)
[beataml_groups] clinical: shape=(942, 95)
[beataml_groups] clinical bridge: subject='dbgap_subject_id', rnaseq='dbgap_rnaseq_sample', dnaseq='dbgap_dnaseq_sample', tp53='TP53', used='used_manuscript_analyses', rna='analysisRnaSeq', exome='analysisExomeSeq'
[beataml_groups] counts key choice: level=rnaseq (overlap rnaseq=671, subject=0)
[beataml_groups] clinical cohort: eligible_subjects=560 (filters applied)
[beataml_groups] mutations: gene_col='symbol' effect_col='variant_classification'
[beataml_groups] mutations: id_col='dbgap_sample_id'
[beataml_groups] TP53 rows=173 protein_altering=173
[beataml_groups] clinical TP53+: n_subjects=81
[beataml_groups] cohort filter: kept 613/710 samples
[beataml_groups] group counts: {'TP53_wt_wes': 552, 'TP53_mut_wes': 58, 'TP53_clinical_only': 3}
[beataml_groups] TP53_mut_wes (rnaseq samples) n=59
[beat

In [25]:
!python paper/scripts/figS_beataml_make_sample_cards.py

[beataml_sample_cards] OK
  groups: /work/paper/source_data/BEATAML_TP53_v1/derived/groups/BEATAML.groups.tsv
  wrote: /work/paper/source_data/BEATAML_TP53_v1/sample_cards/(BEATAML|BEATAML_SWAP).(note|hard).sample_card.json


In [16]:
!Rscript paper/scripts/figS_beataml_deg_rank.R

[beataml_deg_rank] inputs
  counts: /work/paper/source_data/BEATAML_TP53_v1/raw/beataml_waves1to4_counts_dbgap.txt 
  groups: /work/paper/source_data/BEATAML_TP53_v1/derived/groups/BEATAML.groups.tsv 
  include_clinical_only: FALSE 
[beataml_deg_rank] detected sample cols: 610 
[beataml_deg_rank] OK
  gene_col: stable_id 
  wrote: /work/paper/source_data/BEATAML_TP53_v1/derived/rankings/BEATAML.deg_ranking.tsv 
  n_genes: 23658 
  n_samples_used: 610 


In [17]:
!Rscript paper/scripts/figS_beataml_fgsea_to_evidence_table.R

[beataml_fgsea_to_evidence] inputs
  rank: /work/paper/source_data/BEATAML_TP53_v1/derived/rankings/BEATAML.deg_ranking.tsv 
  out : /work/paper/source_data/BEATAML_TP53_v1/evidence_tables/BEATAML.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[beataml_fgsea_to_evidence] stats diagnostics
  stats_len: 23658 
  stats_range: -15.7042250886624,19.9405307130936 
  gene_example: ENSG00000198028,ENSG00000197360,ENSG00000165646,ENSG00000154783,ENSG00000198300,ENSG00000148143,ENSG00000267640,ENSG00000143776,ENSG00000229481,ENSG00000144369 
  gene_id_type: ensembl
 [100%] Downloaded 33730584 bytes...
[beataml_fgsea_to_evidence] msigdbr gene column: ensembl_gene 
[beataml_fgsea_to_evidence] overlap diagnostics
  pathways_with_overlap>=15: 50 / 50 
[beataml_fgsea_to_evidence] OK
  wrote: /work/paper/source_data/BEATAML_TP53_v1/evidence_tables/BEATAML.evidence_table.tsv 
  n_terms: 50 


In [18]:
!head -n 3 /work/paper/source_data/BEATAML_TP53_v1/evidence_tables/BEATAML.evidence_table.tsv

term_id	term_name	source	stat	qval	direction	evidence_genes
HALLMARK_E2F_TARGETS	HALLMARK_E2F_TARGETS	fgsea_msigdb_H_ensembl	3.18099944563885	1.21963657703569e-35	up	ENSG00000162607,ENSG00000134690,ENSG00000142945,ENSG00000188486,ENSG00000177084,ENSG00000108055,ENSG00000148773,ENSG00000164754,ENSG00000100526,ENSG00000164045,ENSG00000166483,ENSG00000166851,ENSG00000138778,ENSG00000111247,ENSG00000126787,ENSG00000186185,ENSG00000130816,ENSG00000090889,ENSG00000024526,ENSG00000105011,ENSG00000129173,ENSG00000012963,ENSG00000149554,ENSG00000139734,ENSG00000164985,ENSG00000089685,ENSG00000117399,ENSG00000035928,ENSG00000142731,ENSG00000170312,ENSG00000131747,ENSG00000099901,ENSG00000156802,ENSG00000157456,ENSG00000122966,ENSG00000101057,ENSG00000077152,ENSG00000165304,ENSG00000135476,ENSG00000171848,ENSG00000100297,ENSG00000161888,ENSG00000161800,ENSG00000111665,ENSG00000119969,ENSG00000152253,ENSG00000087586,ENSG00000181938,ENSG00000138376,ENSG00000013810,ENSG00000253729,ENSG00000136982,EN

In [105]:
!python paper/scripts/run_fig2_pipeline.py \
  --benchmark-id BEATAML_TP53_v1 \
  --conditions BEATAML \
  --variants ours,context_swap,stress \
  --k-claims 50 \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --out-root /work/paper/source_data/BEATAML_TP53_v1/out \
  --context-swap-to BEATAML_SWAP \
  --force

[run_fig2] plan
  benchmark_id: BEATAML_TP53_v1
  conditions: BEATAML
  variants: ours,context_swap,stress
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/BEATAML_TP53_v1/out
  context_swap_to: BEATAML_SWAP
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 15
[run_fig2] (1/15) BEATAML ours gate=hard tau=0.20
[run_fig2] (2/15) BEATAML stress gate=hard tau=0.20
[run_fig2] (3/15) BEATAML context_swap gate=hard tau=0.20
[run_fig2] (4/15) BEATAML ours gate=hard tau=0.40
[run_fig2] (5/15) BEATAML stress gate=hard tau=0.40
[run_fig2] (6/15) BEATAML context_swap gate=hard tau=0.40
[run_fig2] (7/15) BEATAML ours gate=hard tau=0.60
[run_fig2] (8/15) BEATAML stress gate=hard tau=0.60
[run_fig2] (9/15) BEATAML context_swap gate=hard tau=0.60
[run_fig2] (10/15) BEATAML ours gate=hard tau=0.80
[run_fig2] (11/15) BEATAM

In [106]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.80
report   True 152682
audit    True 222253
modules  True 22274
edges    True 197918
terms    True 3238
distilled True 116534
risk     True 1079
card     True 656
meta     True 9436

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'PASS': 39, 'ABSTAIN': 11}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
   benchmark_id                run_id method condition                  comp

In [107]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root "/work/paper/source_data/BEATAML_TP53_v1/out" \
  --out "/work/paper/source_data/BEATAML_TP53_v1/out/risk_coverage.tsv"

[OK] wrote: /work/paper/source_data/BEATAML_TP53_v1/out/risk_coverage.tsv rows=15
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [108]:
import pandas as pd
df = pd.read_csv("/work/paper/source_data/BEATAML_TP53_v1/out/risk_coverage.tsv", sep="\t")
g = df.groupby(["condition","variant"])["tau"].nunique().sort_values()
print(g.head(20))
print("min n_tau =", g.min(), "max n_tau =", g.max(), "rows =", len(df))

condition  variant     
BEATAML    context_swap    5
           ours            5
           stress          5
Name: tau, dtype: int64
min n_tau = 5 max n_tau = 5 rows = 15


In [109]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/BEATAML_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/BEATAML_TP53_v1/fig/Fig_Supp_multipanel_abstain_variants.pdf \
  --figsize 6.1 6.0 \
  --ycol abstain_rate_total \
  --variants ours,context_swap,stress \
  --label-mode variant


In [110]:
!python paper/scripts/fig2_lines_status_by_tau.py \
  --in /work/paper/source_data//BEATAML_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data//BEATAML_TP53_v1/fig/Fig_Supp_abstain_vs_tau_variants.pdf \
  --condition BEATAML \
  --gate-mode hard \
  --variants ours,context_swap,stress \
  --metric abstain \
  --label-mode proposed


In [111]:
!ls /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.80/audit_log.tsv

/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.80/audit_log.tsv


In [112]:
import pandas as pd, sys

p1="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv"
df = pd.read_csv(p1, sep="\t")
print("file:", p1)
print(df["status"].value_counts(dropna=False))
for col in ["reason_code","reason","audit_reason"]:
    if col in df.columns:
        print("\nTop reasons:")
        print(df[df["status"]=="ABSTAIN"][col].value_counts().head(10))
        break
for col in ["survival_min","survival","stability","module_survival","term_survival"]:
    if col in df.columns:
        s = df[col].dropna()
        print("\n", col, "summary:")
        print(s.describe())
        print("counts <0.2, <0.4, <0.6, <0.8, <0.9:",
              [(s < t).sum() for t in [0.2,0.4,0.6,0.8,0.9]])
        break

file: /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv
status
PASS       37
ABSTAIN    13
Name: count, dtype: int64

 term_survival summary:
count    50.000000
mean      0.970938
std       0.047457
min       0.750000
25%       0.957031
50%       0.984375
75%       1.000000
max       1.000000
Name: term_survival, dtype: float64
counts <0.2, <0.4, <0.6, <0.8, <0.9: [np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(3)]


In [113]:
import pandas as pd, sys

p1="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.20/audit_log.tsv"
df = pd.read_csv(p1, sep="\t")
print("file:", p1)
print(df["status"].value_counts(dropna=False))
for col in ["reason_code","reason","audit_reason"]:
    if col in df.columns:
        print("\nTop reasons:")
        print(df[df["status"]=="ABSTAIN"][col].value_counts().head(10))
        break
for col in ["survival_min","survival","stability","module_survival","term_survival"]:
    if col in df.columns:
        s = df[col].dropna()
        print("\n", col, "summary:")
        print(s.describe())
        print("counts <0.2, <0.4, <0.6, <0.8, <0.9:",
              [(s < t).sum() for t in [0.2,0.4,0.6,0.8,0.9]])
        break

file: /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.20/audit_log.tsv
status
PASS       39
ABSTAIN    11
Name: count, dtype: int64

 term_survival summary:
count    50.000000
mean      0.970938
std       0.047457
min       0.750000
25%       0.957031
50%       0.984375
75%       1.000000
max       1.000000
Name: term_survival, dtype: float64
counts <0.2, <0.4, <0.6, <0.8, <0.9: [np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(3)]


In [114]:
import pandas as pd
p="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
sub=df[df["term_survival"]<0.9].copy()
print(sub[["status","term_survival"] + [c for c in ["reason_code","reason","audit_reason","claim_id","term_id","term_name","module_id"] if c in df.columns]])

p="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
cand=[c for c in df.columns if "survival" in c.lower() or "stability" in c.lower()]
print("survival-like columns:", cand)

p="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
for col in ["reason_code","reason","audit_reason"]:
    if col in df.columns:
        print(df[df["status"]=="ABSTAIN"][col].value_counts().head(20))
        break


p20="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.20/audit_log.tsv"
p90="/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv"

a=pd.read_csv(p20, sep="\t")
b=pd.read_csv(p90, sep="\t")

keep=["claim_id","status","term_id","term_survival","term_survival_agg","stability_ok","stability_scope"]
keep=[c for c in keep if c in a.columns and c in b.columns]

m=a[keep].merge(b[keep], on="claim_id", how="outer", suffixes=("_20","_90"))

# 1) statusが変わったやつ
chg=m[m["status_20"]!=m["status_90"]].copy()
print("Status changed:", len(chg))
print(chg.sort_values(["status_20","status_90"]).head(50).to_string(index=False))

# 2) stability_okが変わったやつ
if "stability_ok_20" in m.columns:
    chg2=m[m["stability_ok_20"]!=m["stability_ok_90"]].copy()
    print("\nStability_ok changed:", len(chg2))
    print(chg2.head(50).to_string(index=False))

     status  term_survival        claim_id  \
34  ABSTAIN       0.843750  c_795815b409e8   
35  ABSTAIN       0.750000  c_cf13126e115b   
43  ABSTAIN       0.890625  c_a82594b21128   

                                     term_id  \
34  HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY   
35              HALLMARK_PANCREAS_BETA_CELLS   
43       HALLMARK_WNT_BETA_CATENIN_SIGNALING   

                                   term_name      module_id  
34  HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY  Meb3d28316a2b  
35              HALLMARK_PANCREAS_BETA_CELLS  M04098975cc06  
43       HALLMARK_WNT_BETA_CATENIN_SIGNALING  M20916fa39560  
survival-like columns: ['term_survival', 'stability_ok', 'term_survival_agg', 'stability_scope']
Status changed: 2
      claim_id status_20                               term_id_20  term_survival_20  term_survival_agg_20  stability_ok_20 stability_scope_20 status_90                               term_id_90  term_survival_90  term_survival_agg_90  stability_ok_90 stability

In [132]:
!python paper/scripts/figS_plot_abstain_reasons.py \
  --audit-logs \
    /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.90/audit_log.tsv \
    /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/context_swap/gate_hard/tau_0.90/audit_log.tsv \
    /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/stress/gate_hard/tau_0.90/audit_log.tsv \
  --out /work/paper/source_data/BEATAML_TP53_v1/fig/Fig_Supp_abstain_reasons_tau0.90.pdf \
  --title "" \
  --topk 5
